In [2]:
from google.colab import drive
drive.mount('/content/Mydrive/')

Mounted at /content/Mydrive/


Preprocesamiento de imágenes

In [3]:
import cv2
import torch
from PIL import Image
from torchvision import transforms

# Transformaciones para adaptar las imágenes al modelo ViT
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar las imágenes a 224x224
    transforms.ToTensor(),  # Convertir las imágenes a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización estándar
])

def divide_into_patches(image_path, patch_size=224):
    """
    Divide una imagen grande en parches de tamaño patch_size x patch_size.

    Parámetros:
        - image_path (str): ruta de la imagen original (aumentada a x40)
        - patch_size (int): tamaño de los parches que se generarán (por defecto 224x224)

    Devuelve:
        - patches (list): una lista con los parches (subimágenes) generados
    """
    # Cargar la imagen
    img = cv2.imread(image_path)

    # Obtener las dimensiones de la imagen
    height, width, _ = img.shape

    # Lista para almacenar los parches
    patches = []

    # Dividir la imagen en parches de patch_size x patch_size
    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            patch = img[y:y + patch_size, x:x + patch_size]
            # Asegurarse de que el parche sea exactamente del tamaño especificado
            if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
                patches.append(patch)

    return patches

# Ejemplo de uso:
image_path = '/content/Mydrive/MyDrive/IH/9175/1/9175_idx5_x1801_y251_class1.png'
patches = divide_into_patches(image_path)

# Preprocesar los parches utilizando las transformaciones para ViT
patches_tensor = [data_transforms(Image.fromarray(patch)) for patch in patches]

print(f"Total de parches generados: {len(patches_tensor)}")


Total de parches generados: 0


Modelo Vision Transformer con DINO

In [9]:
import torch.nn as nn
import torch

# Cargar el modelo ViT-B/8 preentrenado con DINO desde torch.hub
model = torch.hub.load('facebookresearch/dino:main', 'dino_vitb8')

# Enviar el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Crear un tensor de ejemplo para obtener el tamaño de la salida del modelo
# El modelo ViT espera una entrada de tamaño (batch_size, 3, 224, 224)
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Pasar el tensor de ejemplo a través del modelo para ver la salida
model.eval()  # Poner el modelo en modo evaluación
with torch.no_grad():
    output = model(dummy_input)

# Obtener el tamaño de las características de la capa final (número de características de salida)
num_ftrs = output.shape[1]  # El tamaño de la salida es (batch_size, num_ftrs)

# Cambiar la última capa del modelo (head) para que clasifique 2 clases (IDC negativo o IDC positivo)
model.head = nn.Linear(num_ftrs, 2)  # 2 clases: IDC negativo o IDC positivo

print(f"Número de características de la última capa: {num_ftrs}")
print(f"Modelo Vision Transformer (ViT-B/8) con DINO cargado exitosamente en {device}.")


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


Número de características de la última capa: 768
Modelo Vision Transformer (ViT-B/8) con DINO cargado exitosamente en cpu.


División del conjunto de datos

In [10]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, Y_train, Y_test = train_test_split(patches_tensor, Y_tensor, test_size=0.2, random_state=42)

# Convertir a tensores y mover los datos a GPU (si está disponible)
X_train, X_test = torch.stack(X_train).to(device), torch.stack(X_test).to(device)
Y_train, Y_test = torch.tensor(Y_train).to(device), torch.tensor(Y_test).to(device)

# Mostrar las dimensiones del conjunto de datos
print(f"Datos de entrenamiento: {X_train.shape}, Etiquetas de entrenamiento: {Y_train.shape}")
print(f"Datos de prueba: {X_test.shape}, Etiquetas de prueba: {Y_test.shape}")


NameError: name 'Y_tensor' is not defined

Entrenamiento del modelo

In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Crear dataloaders para manejar los lotes de entrenamiento y prueba
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definir el optimizador (Adam) y la función de pérdida (CrossEntropyLoss para clasificación binaria)
optimizer = Adam(model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss()

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    """
    Función para entrenar el modelo Vision Transformer.

    Parámetros:
        - model: modelo de Vision Transformer (ViT) preentrenado con DINO.
        - train_loader: dataloader que contiene los datos de entrenamiento.
        - criterion: función de pérdida (CrossEntropyLoss).
        - optimizer: optimizador (Adam).
        - num_epochs: número de épocas para entrenar.

    Devuelve:
        - model: modelo entrenado.
    """
    model.train()  # Poner el modelo en modo de entrenamiento

    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  # Reiniciar los gradientes acumulados

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass y optimización
            loss.backward()
            optimizer.step()

            # Sumar la pérdida y las predicciones correctas
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calcular la pérdida y precisión por época
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f'Epoch {epoch}/{num_epochs-1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

    return model

# Entrenar el modelo
model = train_model(model, train_loader, criterion, optimizer, num_epochs=10)


Evaluación del modelo

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_model(model, test_loader):
    """
    Función para evaluar el modelo en el conjunto de prueba.

    Parámetros:
        - model: modelo Vision Transformer entrenado.
        - test_loader: dataloader que contiene los datos de prueba.

    Imprime:
        - Precisión del modelo, reporte de clasificación y matriz de confusión.
    """
    model.eval()  # Poner el modelo en modo de evaluación
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Guardar las predicciones y etiquetas reales
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calcular la precisión
    accuracy = accuracy_score(true_labels, predictions)
    print(f'Precisión del modelo: {accuracy:.4f}')

    # Imprimir el reporte de clasificación
    print('Reporte de Clasificación:')
    print(classification_report(true_labels, predictions))

    # Imprimir la matriz de confusión
    print('Matriz de Confusión:')
    cm = confusion_matrix(true_labels, predictions)
    print(cm)

# Evaluar el modelo en el conjunto de prueba
evaluate_model(model, test_loader)
